## Binding
If we have a function pointer, we can invoke the corresponding function using the function pointer and by giving it the required parameters. Bind takes the callable object as its first argument and performs a partial function call. It binds the remaining arguments to this callable object and returns a new callable object.
```
auto match_cat = std::bind(match, "cat");  //bind's "cat" to match callable object and returns match_cat callable object
match_cat(); //will be called as match("cat")
```
Bind also alows for argument forwarding. match_cat will take the arguments and forward them to match. To do this we have placeholders, _1 will be replaced by arg1, _2 by agr2 etc.
```
auto match_cat = std::bind(match, std::placeholders::_1, "cat"); //bind "cat" as the second argument
match_cat("dog"); //will be called as match("dog", "cat")
```
We can also forward aguments in different order
```
auto match_cat = std::bind(match, std::palceholders::_2, "cat", std::palceholders::_1);
match_cat(arg1, arg2); //will be called as match(arg2, "cat", arg1);
```
Bind is equivalent to lambda with a local variable
```
auto match_cat = [species="cat"](const std::string &animal)
{
    return match(animal, species);
}
```

## Callable Objects
A callable objects is anything that supports the () operator. These are pointer to non member functions, functors(classes which define the () operator), lambda expressions(functors generated by the compiler) and objects returned from bind(). All these different callable objects can be saved into std::function, hence these callable objects are interoperable. std::function class has a private member which stores the callable object, its a template class and the template parameter is the callable objects signature.
```
std::function<void(const string&)> match_ptr;
```
We can use std::function as a function argument and then call that function with any callable object with that signature. std::function is implemented using inheritance for the varies callable objects mentioned above, hence it can be slow due to indirection. For this reason storing a callable object in a auto variable is best, use std::function only if you need a polymorphic function object.

## Member Function Pointers
C++ allows us to define a pointer to a member function of a class. Their syntax and using them is more complicated than global function pointers, we need to provide the object that they are called on. Member function pointers cannot be called directly, they must be dereferenced(that means they are not callable objects).
```
class test
{
public:
    void func(int a, const std::string &b);
};

void (test::*pfunc)(int, const std::string&); //define pfunc as a pointer to a member function of test
pfunc = &test::func; //make pfunc point to the member function

//to call
test t; //create the object
(t.*pfunc)(42, "Hello"s); //get the pointer as the member of the object, dereference it and call
```
With modern C++ we can use auto instead of defining pfunc pointer. C++ also has std::mem_fn function to convert member function pointers to callable objects. when we use this callable object to invoke the function we have to pass the object too.
```
auto f = std::mem_fn(pfunc);

f(t, 42, "Hello"s); //pass the 'this' object as the first argument
```
We can also use std::bind with member function pointers.
```
test t;
auto f = std::bind(&test::func, &t, std::placeholders::_1, std::placeholders::_2);
f(42, "Hello"s);
```
We can use member function pointers when we want to start a thread in a class which runs one of the member functions in that class or to register member functions to a eventing system.